# 참고자료

https://keras.io/examples/vision/oxford_pets_image_segmentation/

# 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/dataset/dataset.zip

# Chapter 1: Semantic Segmentation 이해하기

- https://fullstackdeeplearning.com/spring2021/lecture-2b/
- https://vision-explorer.allenai.org
- Classification, Object Detection, Semantic Segmentation의 차이 (입력과 출력이 무엇일까요?)

# Chapter 2: QGIS로 데이터셋 만들기

- QGIS로 road_polygon.shp, orthophoto_5179.tif 열기
- 래스터화 (벡터를 래스터로)
- 타일 재생성 (-of PNG)

# Chapter 3: TensorFlow로 학습시키기

필요한 패키지 로드하기

In [ ]:
import os
from pathlib import Path

from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import optimizers, losses, layers, Input, Model

데이터셋 정의하기

In [ ]:
def _replace_folder_name(src_img_fpath, folder_name):
    dst_img_fpath = bytes.decode(src_img_fpath.numpy())
    dst_img_fpath = Path(dst_img_fpath).parts
    dst_img_fpath = list(dst_img_fpath)
    dst_img_fpath[-2] = bytes.decode(folder_name.numpy())
    dst_img_fpath = str(Path(*dst_img_fpath))
    return dst_img_fpath


def _parse_image(a_fpath, target_size, multiclass):
    # Create file path for B and cmap using a_fpath
    c_fpath = tf.py_function(_replace_folder_name, [a_fpath, "cmap"], tf.string)

    # Read files
    a_img = tf.io.read_file(a_fpath)
    c_img = tf.io.read_file(c_fpath)

    # Decode files
    a_img = tf.image.decode_png(a_img, channels=3)
    c_img = tf.image.decode_png(c_img, channels=1)

    # Resize images
    a_img = tf.image.resize(a_img, target_size)
    c_img = tf.image.resize(c_img, target_size)

    # Normalize images [0, 255] to [-1.0, 1.0]
    a_img = tf.cast(a_img, tf.float32) / 127.5 - 1

    # 255로 나눈 후 int8로 타입캐스팅 (int여야 SparseCatecoricalCrossentropy 적용 시 문제 없음)
    c_img = tf.cast(c_img / 255, tf.int8)

    return a_img, c_img


def get_dataset(
    dataset_dir, batch_size, target_size=(256, 256), shuffle=True, multiclass=False
):
    a_file_pattern = os.path.join(dataset_dir, "A/*.png")
    ds = tf.data.Dataset.list_files(file_pattern=a_file_pattern, shuffle=shuffle)
    ds = ds.map(lambda x: _parse_image(x, target_size, multiclass))
    ds = ds.batch(batch_size)
    return ds

모델 정의하기

In [ ]:
def get_model(img_size, num_classes):
    inputs = Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = Model(inputs, outputs)
    return model

In [ ]:
model = get_model((256, 256,), 2)
model.summary()

In [ ]:
ds = get_dataset("dataset", 4)

In [ ]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss=losses.SparseCategoricalCrossentropy(),
)

In [ ]:
model.fit(ds, epochs=10)

In [ ]:
result = model.predict(ds)

In [ ]:
plt.imshow(result[133][:, :, 1])